In [154]:
# Importamos todo lo que vamos a utilizar
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag
import re
from nltk.corpus import stopwords
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize, word_tokenize
from sklearn.ensemble import RandomForestClassifier
import gensim
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

In [162]:
# Cargamos el dataset
df = pd.read_csv('reddit_database_sentiment.csv', delimiter=';', low_memory=False)

En este módulo realizaremos varios métodos para determinar el sentimiento de un determinado post

### Un método basado en lexicons

El primer método será basándonos en lexicons. Para ello, utilizaremos el corpus SentiWordNet que proporciona el sentimiento positivo, negativo y objetivo de una palabra determinada y pudiendo especificar su etiqueta. Sumaremos el sentimiento de cada palabra si aparece en este diccionario. La suma total positiva, negativa y objetiva, es el valor de sentimiento del post.

Para ello, necesitaremos realizar varias funciones previas para que analizen el valor de cada palabra y su categoría gramatical.

In [163]:
# La siguiente función estandariza la etiqueta 
def convert_tag(pos_tag):
    if pos_tag.startswith('JJ'):
        return wn.ADJ
    elif pos_tag.startswith('NN'):
        return wn.NOUN
    elif pos_tag.startswith('RB'):
        return wn.ADV
    elif pos_tag.startswith('VB') or pos_tag.startswith('MD'):
        return wn.VERB
    return None

In [164]:
# Esta función va a obtener el sentimiento de una palabra en base a su etiqueta
def get_sentiment(word, tag):
    wn_tag = convert_tag(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV): 
        return []
    lemma = wn.morphy(word, pos=wn_tag)
    if lemma is None:
        return []
    synsets = wn.synsets(lemma, pos=wn_tag)
    if not synsets:
        return []
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())
    return [swn_synset.pos_score(), swn_synset.neg_score(), swn_synset.obj_score()]

### Método basado en palabras únicas en textos de cada tipo de sentimiento

Para el segundo método, crearemos un algoritmo que sea entrenado con un conjunto de datos y sus etiquetas en base a la frecuencia que aparece en un post. Dado un post, extraigo el vocabulario y la frecuencia de este en un diccionario. Si el post es positivo lo añado al vocabulario positivo y si es un post negativo lo añado al vocabuario negativo.
Para la clasificación, por cada palabra veo si está en el diccionario positivo y negativo, hallo su frecuencia y lo sumo a la nota que corresponda (positiva o negativa). Comparo que nota es mayor y devuelvo si es mayor la positiva el post es positivo y si es mayor la negativa el post es negativo.

In [165]:
def entrenamiento(set_post, set_sentiment):
    positive_vocab = []
    negative_vocab = []
    for i in set_post.index:
        post = set_post[i]
        if set_sentiment[i] == 'POSITIVE':
            positive_vocab.extend(post)
        elif set_sentiment[i] == 'NEGATIVE':
            negative_vocab.extend(post)
    # Construcción de la distribución de frecuencia
    fdist_pos = FreqDist(positive_vocab)
    fdist_neg = FreqDist(negative_vocab)
    
    return fdist_pos, fdist_neg

In [166]:
# Obtener el set de entrenamiento y set de test para los posts y sus etiquetas.
X_train, y_train, X_test, y_test = train_test_split(df['post'].fillna(''), df['sentiment'].fillna(''), stratify=df['sentiment'].fillna(''), test_size=.3, random_state=1)

In [167]:
# Diccionarios de vocabulario positivo y su frecuencia y vocabulario negativo y su frecuencia
pos, neg = entrenamiento(X_train, X_test)

### Método word embeddings + algoritmo de ML de clasificación

Para el último método implementaremos la técnica de Word embeddings y el algoritmo de ML RandomForestClassifier. La técnica de Word embedding consiste en convertir cada palabra en un vector. Como necesitamos representar cada post como un vector, hemos obtenido los vectores de cada palabra y hemos realizado el promedio de estos. Por lo tanto, el vector que representa un post es el vector promedio de los vectores de cada una de sus palabras.

Posteriormente hemos transformado el set de entrenamiento en 0 para sentimiento negativo y en 1 para sentimiento positivo.

Una vez tenemos el conjunto de vectores que representan cada post de entrenamiento y un vector con la etiqueta de cada uno de estos post, entrenamos el modelo.

In [168]:
vectores_post = [] # Lista de vectores(posts)

for post in X_train: # Recorremos cada post
    sentences = sent_tokenize(post)
    tokenized_sentences = [word_tokenize(sent.lower()) for sent in sentences]
    vectores = []

    # Obtenemos los vectores de sus palabras
    model = Word2Vec(sentences=tokenized_sentences, vector_size=50, window=5, sg=1, min_count=1, workers=4)
    # Añadimos cada vector palabra a la lista de vectores
    model.build_vocab(tokenized_sentences)  # Construye el vocabulario a partir de las oraciones
    
    # Entrenar el modelo
    model.train(tokenized_sentences, total_examples=model.corpus_count, epochs=10)
    for palabra in word_tokenize(post.lower()):
        if palabra in model.wv:
            vectores.append(model.wv[palabra])
    # Obtenemos el promedio (vector post)
    if len(vectores) > 0:
        vector_promedio = np.mean(vectores, axis=0)
    else:
        vector_promedio = np.zeros(model.vector_size)
    # Añadimos el vector del post analizado
    vectores_post.append(vector_promedio)

In [169]:
# Conversión de las etiquetas de sentimiento a valor numérico
y = X_test.apply(lambda x: 1 if x=='POSITIVE' else 0)

In [170]:
# Creación del modelo y entrenamiento
clf=RandomForestClassifier(n_estimators=10)
clf.fit(vectores_post, y)

RandomForestClassifier(n_estimators=10)

In [188]:
def sentiment_analysis(text:str):
    # Basado en lexicons
    pos_tags = pos_tag(word_tokenize(text))
    pos_score = neg_score = obj_score = 0
    for w, t in pos_tags:
        sentiment = get_sentiment(w, t)
        if sentiment:
            pos_score += sentiment[0]
            neg_score += sentiment[1]
            obj_score += sentiment[2]
    resul_metodo1 = pos_score, neg_score, obj_score
    
    # Palabras únicas en textos de ambos sentimientos
    resul_metodo2 = ''
    pos_score = 0
    neg_score = 0
    for palabra in text:
        pos_score += pos[palabra]
        neg_score += neg[palabra]
    if pos_score > neg_score:
        resul_metodo2 = 'POSITIVE'
    else:
        resul_metodo2 = 'NEGATIVE'

    # Word embeddings
    # Conversión del texto a vector
    sentences = sent_tokenize(text)
    tokenized_sentences = [word_tokenize(sent.lower()) for sent in sentences]
    vectores = []
    # Obtenemos los vectores de sus palabras
    model = Word2Vec(sentences=tokenized_sentences, vector_size=50, window=5, sg=1, min_count=1, workers=4)
    # Añadimos cada vector palabra a la lista de vectores
    for palabra in word_tokenize(post.lower()):
        if palabra in model.wv:
            vectores.append(model.wv[palabra])
    # Obtenemos el promedio (vector post)
    if len(vectores) > 0:
        vector_promedio = np.mean(vectores, axis=0)
    else:
        vector_promedio = np.zeros(model.vector_size)

    # Aplicación del vector promedio obtenido al modelo clf
    resul_metodo3 = clf.predict([vector_promedio])
    if resul_metodo3[0] == '0':
        resul_metodo3 = 'NEGATIVE'
    else:
        resul_metodo3 = 'POSITIVE'

    print(resul_metodo1)
    print(resul_metodo2)
    print(resul_metodo3)

In [181]:
print(y_train)

1877    http://i.imgur.com/QJd0eBH.png\n\nFFS\n\nAny o...
1921    I'm trying to test a form submission within a ...
1844    I'm still very new to GTM so bear with me. \n\...
1402    Is this scenario possible?\n\nPage URLs are ch...
1193    Hey r/analytics, \n\nAre there any high qualit...
                              ...                        
1786    I got a big uptick in facebook traffic, looks ...
1956    Hey,\n\nI have an issue with AdWords report (a...
1677    Based on the click numbers our agency is repor...
1974    I got 3 different sites I monitor a lot. Each ...
1101    Investors are always asking us what is our rec...
Name: post, Length: 300, dtype: object


In [189]:
print('Post:', y_train[1877])
print()
print('Etiqueta real: ', y_test[1877])
print()
print('Etiquetas de la función:', sentiment_analysis(y_train[1877]))

Post: http://i.imgur.com/QJd0eBH.png

FFS

Any of you tried evading this by using 4-5 accounts on your site based on a cookie value? I can reduce the amount of hits by about 1.5 million but they want me to do things that would increase it by about 6 million.  

There is no budget for 360. 

I've dealt with this before by creating a cookie/localStorage/beforeunload method of merging hits together.... but this requires so much planning and testing, I just cba anymore. 

Etiqueta real:  NEGATIVE

(0.0, 0.0, 18.0)
NEGATIVE
POSITIVE
Etiquetas de la función: None
